<a href="https://colab.research.google.com/github/pro-anish/Bert-Legal/blob/main/Classifying_sentences_of_a_case_document_into_rhetorical_roles_using_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import glob
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

In [ ]:
import torch

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers

     |████████████████████████████████| 3.5 MB 3.9 MB/s 
     |████████████████████████████████| 6.8 MB 35.2 MB/s 
     |████████████████████████████████| 67 kB 5.2 MB/s 
     |████████████████████████████████| 596 kB 49.8 MB/s 
     |████████████████████████████████| 895 kB 51.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
dirpath = '/content/drive/MyDrive/text'
output = '/content/drive/MyDrive/text/combined.csv'

csvout_lst = []
files = [os.path.join(dirpath, fname) for fname in os.listdir(dirpath)]
(files.remove('/content/drive/MyDrive/text/combined.csv'))
try:
  (files.remove('/content/drive/MyDrive/text/.ipynb_checkpoints'))
  
except:
  None

In [ ]:
!pip install torchtext

In [ ]:
for filename in sorted(files):
    data = pd.read_csv(filename, sep='\n', index_col=0, header=None)
    csvout_lst.append(data)

pd.concat(csvout_lst).to_csv(output)

In [ ]:
data= pd.read_csv('/content/drive/MyDrive/text/combined.csv')
data.columns = ['sentence']
data

,sentence
0,"One Lakshminarayana Iyer, a Hindu Brahmin, who..."
1,Ramalakshmi had married the plaintiff and had ...
2,"They were all alive in December, 1924, when La..."
3,Before his death he executed a will on 16th No...
4,By this will he gave the following directions ...
...,...
9375,We find no infirmity in the impugned order pas...
9376,"Hence,this appeal has no merit which is accord..."
9377,"Before parting with the order,we must make it ..."
9378,The parties may prosecute their remedies in Ci...


In [ ]:
data[['sentence','label']]=data['sentence'].str.split('\t', expand=True)


In [ ]:
data

,sentence,label
0,"One Lakshminarayana Iyer, a Hindu Brahmin, who...",Facts
1,Ramalakshmi had married the plaintiff and had ...,Facts
2,"They were all alive in December, 1924, when La...",Facts
3,Before his death he executed a will on 16th No...,Ratio of the decision
4,By this will he gave the following directions ...,Facts
...,...,...
9375,We find no infirmity in the impugned order pas...,Ruling by Present Court
9376,"Hence,this appeal has no merit which is accord...",Ruling by Present Court
9377,"Before parting with the order,we must make it ...",Ruling by Present Court
9378,The parties may prosecute their remedies in Ci...,Ruling by Present Court


In [ ]:
sentences= data.sentence.values
labels= data.label.values

In [ ]:
sentences

array(['One Lakshminarayana Iyer, a Hindu Brahmin, who owned considerable properties in the Tirunelveli district, died on 13th December, 1924, leaving him surviving a widow Ranganayaki, and a married daughter Ramalakshmi',
       'Ramalakshmi had married the plaintiff and had a number of children from him',
       'They were all alive in December, 1924, when Lakshminarayana died',
       ...,
       'Before parting with the order,we must make it clear that in view of the request made by the appellant before the High Court not to enter into the merit of the case since the party may prosecute their remedies in the Civil Court for adjudication,we have not expressed any opinion with regard to the merit of the case of the parties',
       'The parties may prosecute their remedies in Civil Court in accordance with law',
       'Petition dismissed'], dtype=object)

In [ ]:

possible_labels = data.label.unique()

label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
label_dict


{'Argument': 3,
 'Facts': 0,
 'Precedent': 5,
 'Ratio of the decision': 1,
 'Ruling by Lower Court': 2,
 'Ruling by Present Court': 4,
 'Statute': 6}

In [ ]:

data['labels'] = data.label.replace(label_dict)
data.groupby(['label']).count()


,sentence,labels
label,,
Argument,845,845
Facts,2219,2219
Precedent,1468,1468
Ratio of the decision,3624,3624
Ruling by Lower Court,316,316
Ruling by Present Court,262,262
Statute,646,646


In [ ]:
import torchtext.data

import torch
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification

In [ ]:
from sklearn.model_selection import train_test_split

sample_size = int(len(data)*1)
sampledf = data.sample(sample_size)


X_train, X_val, y_train, y_val = train_test_split(sampledf.index.values, 
                                                  sampledf.label.values, 
                                                  test_size=0.1, 
                                                  random_state=42, 
                                                  stratify=sampledf.label.values)

sampledf['data_type'] = ['not_set']*sampledf.shape[0]

sampledf.loc[X_train, 'data_type'] = 'train'
sampledf.loc[X_val, 'data_type'] = 'val'
data=sampledf
sampledf.groupby(['label', 'labels', 'data_type']).count()
df=data


In [ ]:
print(y_train)
X_train

['Ratio of the decision' 'Ratio of the decision' 'Facts' ... 'Precedent'
 'Ratio of the decision' 'Facts']


array([5714, 3863, 8222, ..., 8136, 7728,  338])

In [ ]:
tokenizer = BertTokenizer.from_pretrained('nlpaueb/legal-bert-base-uncased', 
                                          do_lower_case=True)
                                          
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].sentence.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].sentence.values, 
    add_special_tokens=True, 
    return_attention_mask=True, 
    pad_to_max_length=True, 
    return_tensors='pt'
)


input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].labels.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].labels.values)

dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/217k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/0.99k [00:00<?, ?B/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2257: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
labels_train

tensor([3, 1, 1,  ..., 5, 3, 0])

In [ ]:
encoded_data_train

{'input_ids': tensor([[ 101,  248, 5365,  ...,    0,    0,    0],
        [ 101,  300,  212,  ...,    0,    0,    0],
        [ 101,  207, 1222,  ...,    0,    0,    0],
        ...,
        [ 101,  233,  246,  ...,    0,    0,    0],
        [ 101,  264,  210,  ...,    0,    0,    0],
        [ 101,  676,  117,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [ ]:
model = BertForSequenceClassification.from_pretrained("nlpaueb/legal-bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)
model.cuda()

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at nlpaueb/legal-bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification wer

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [ ]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

batch_size = 8

dataloader_train = DataLoader(dataset_train, 
                              sampler=RandomSampler(dataset_train), 
                              batch_size=batch_size)

dataloader_validation = DataLoader(dataset_val, 
                                   sampler=SequentialSampler(dataset_val), 
                                   batch_size=batch_size)

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model.parameters(),
                  lr=6e-6, 
                  eps=1e-8)
                  
epochs = 5

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use thePyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

In [ ]:
torch.cuda.empty_cache()
torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |     839 MB |    5730 MB |   79189 GB |   79188 GB |\n|       from large pool |     838 MB |    5728 MB |   79160 GB |   79159 GB |\n|       from small pool |       1 MB |      16 MB |      28 GB |      28 GB |\n|---------------------------------------------------------------------------|\n| Active memory         |     839 MB |    5730 MB |   79189 GB |   79188 GB |\n|       from large pool |     838 MB |    5728 MB |

In [ ]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals
    
for epoch in tqdm(range(1, epochs+1)):          
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()       #set the gradients to zero before starting to do backpropragation
        batch = tuple(b.to(device) for b in batch)
        

        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       

        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'/content/drive/MyDrive/finetuned_BERT_epoch_try_2{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/1056 [00:00<?, ?it/s]


Epoch 1
Training loss: 0.510462989109907
Validation loss: 1.369875606370427
F1 Score (Weighted): 0.6035035113894524


Epoch 2:   0%|          | 0/1056 [00:00<?, ?it/s]


Epoch 2
Training loss: 0.432814248214885
Validation loss: 1.4340057182362524
F1 Score (Weighted): 0.6226540463613764


...


Epoch 3
Training loss: 0.3665433705468648
Validation loss: 1.5963727230111422
F1 Score (Weighted): 0.6109757527217556


Epoch 4:   0%|          | 0/1056 [00:00<?, ?it/s]


Epoch 4
Training loss: 0.33170879492139316
Validation loss: 1.6695121372655286
F1 Score (Weighted): 0.6059689919673177


Epoch 5:   0%|          | 0/1056 [00:00<?, ?it/s]


Epoch 5
Training loss: 0.39493823957110924
Validation loss: 1.6497008556152803
F1 Score (Weighted): 0.6130547512216752


In [ ]:
model.load_state_dict(torch.load('/content/drive/MyDrive/finetuned_BERT_epoch_try_24.model', map_location=torch.device('cuda')))

_, predictions, true_vals = evaluate(dataloader_validation)
accuracy_per_class(predictions, true_vals)

Class: Facts
Accuracy: 144/222

Class: Ratio of the decision
Accuracy: 252/362

Class: Ruling by Lower Court
Accuracy: 5/32

Class: Argument
Accuracy: 44/84

Class: Ruling by Present Court
Accuracy: 16/26

Class: Precedent
Accuracy: 69/147

Class: Statute
Accuracy: 44/65

